This notebook uses a set of functions and module within the bispy package to interact with the NatureServe API.

NatureServe species documents are retrieved and cached based on valid/accepted ITIS species names and the invalid/not accepted ITIS species names.

In [1]:
#Import needed packages
import json
import bispy
from IPython.display import display
from joblib import Parallel, delayed

natureserve = bispy.natureserve.Natureserve()
bis_utils = bispy.bis.Utils()

In [2]:
# Open up the cached explore ITIS list with species names from ITIS Exploration.ipynb
with open("cache/itis_explore.json", "r") as f:
    itis_explore = json.loads(f.read())

In [3]:
# Use joblib to run multiple requests for NatureServe records in parallel via ITIS scientific names
natureserve_results = Parallel(n_jobs=8)(delayed(natureserve.search)(name) for name in [r["scientific_name"] for r in itis_explore])
len(natureserve_results)

170

In [4]:
natureserve_results

[{'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-27T20:29:26.135883',
   'status_message': 'Single Match',
   'api': 'https://services.natureserve.org/idd/rest/v1/nationalSpecies/summary/nameSearch?nationCode=US&name=Oncorhynchus clarkii pleuriticus'},
  'parameters': {'Scientific Name': 'Oncorhynchus clarkii pleuriticus',
   'Name Source': None},
  'NatureServe Species': {'@uid': 'ELEMENT_NATIONAL.2.166421',
   '@type': 'Animal',
   'jurisdictionNationName': {'@code': 'US', '#text': 'UNITED STATES'},
   'nationalScientificName': {'unformattedName': 'Oncorhynchus clarkii pleuriticus',
    'formattedName': '<i>Oncorhynchus clarkii pleuriticus</i>',
    'nomenclaturalAuthor': '(Cope, 1872)'},
   'nationalConservationStatus': {'@lastChangedDate': '1996-12-05',
    '@lastReviewedDate': '1996-12-05',
    '#text': 'N2N3'},
   'roundedNationalConservationStatus': 'N2',
   'nationalCommonName': 'Colorado River Cutthroat Trout',
   'natureServeGlobalConcept': {'@uid'

In [5]:
# Filter to give just cases where NatureServe species names matched with ITIS species names
success_natureserve_results = [i for i in natureserve_results if i["processing_metadata"]["status"] == "success"]
len(success_natureserve_results)

138

In [6]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/natureserve.json", success_natureserve_results))

{'Doc Cache File': 'cache/natureserve.json',
 'Number of Documents in Cache': 138,
 'Document Number 24': {'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-27T20:29:26.135883',
   'status_message': 'Single Match',
   'api': 'https://services.natureserve.org/idd/rest/v1/nationalSpecies/summary/nameSearch?nationCode=US&name=Hesperostipa comata'},
  'parameters': {'Scientific Name': 'Hesperostipa comata',
   'Name Source': None},
  'NatureServe Species': {'@uid': 'ELEMENT_NATIONAL.2.246060',
   '@type': 'Plant',
   'jurisdictionNationName': {'@code': 'US', '#text': 'UNITED STATES'},
   'nationalScientificName': {'unformattedName': 'Hesperostipa comata',
    'formattedName': '<i>Hesperostipa comata</i>',
    'nomenclaturalAuthor': '(Trin. & Rupr.) Barkworth'},
   'nationalConservationStatus': {'@lastChangedDate': '1993-08-02',
    '#text': 'NNR'},
   'roundedNationalConservationStatus': 'NNR',
   'natureServeGlobalConcept': {'@uid': 'ELEMENT_GLOBAL.2.133544',
    

Check to see if any invalid/not accepted ITIS species names matched with NatureServe records

In [7]:
# Open up the cached updated ITIS list with invalid/not accepted species names from ITIS Exploration.ipynb
with open("cache/invalid_itis.json", "r") as f:
    invalid_itis = json.loads(f.read())

In [8]:
# Use joblib to run multiple requests for NatureServe documents in parallel via the invalid/not accepted ITIS scientific names
invalid_result = Parallel(n_jobs=8)(delayed(natureserve.search)(name) for name in [r["scientific_name"] for r in invalid_itis]) 

In [9]:
# Filter to give just cases where NatureServe species name matched with invalid/not accepted ITIS names
invalid_result=[i for i in invalid_result if i['processing_metadata']['status'] == 'success']
len(invalid_result)

2

In [10]:
#Display result to identify the NatureServe record that matched the invalid/not accepted ITIS name
invalid_result

[{'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-27T20:29:26.135883',
   'status_message': 'Single Match',
   'api': 'https://services.natureserve.org/idd/rest/v1/nationalSpecies/summary/nameSearch?nationCode=US&name=Clupea pallasi'},
  'parameters': {'Scientific Name': 'Clupea pallasi', 'Name Source': None},
  'NatureServe Species': {'@uid': 'ELEMENT_NATIONAL.2.164813',
   '@type': 'Animal',
   'jurisdictionNationName': {'@code': 'US', '#text': 'UNITED STATES'},
   'nationalScientificName': {'unformattedName': 'Clupea pallasii',
    'formattedName': '<i>Clupea pallasii</i>',
    'nomenclaturalAuthor': 'Valenciennes, 1847'},
   'nationalConservationStatus': {'@lastChangedDate': '1996-12-05',
    '@lastReviewedDate': '1996-12-05',
    '#text': 'N5'},
   'roundedNationalConservationStatus': 'N5',
   'nationalCommonName': 'Pacific Herring',
   'natureServeGlobalConcept': {'@uid': 'ELEMENT_GLOBAL.2.105816',
    'classificationStatus': 'Standard',
    'natureServ